In [1]:
# Dependencies
import pandas as pd
import numpy as np

In [2]:
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name"])

school_data_complete.head()

,Student ID,student_name,gender,year,school_name,reading_score,maths_score,School ID,type,size,budget
0,0,Paul Bradley,M,9,Huang High School,96,94,0,Government,2917,1910635
1,1,Victor Smith,M,12,Huang High School,90,43,0,Government,2917,1910635
2,2,Kevin Rodriguez,M,12,Huang High School,41,76,0,Government,2917,1910635
3,3,Richard Scott,M,12,Huang High School,89,86,0,Government,2917,1910635
4,4,Bonnie Ray,F,9,Huang High School,87,69,0,Government,2917,1910635


In [3]:
# Calculate the percent passing math and reading
passing_math = student_data[student_data['maths_score'] >= 70]

passing_math_rate = (len(passing_math) / len(student_data)) * 100

passing_reading = student_data[student_data['reading_score'] >= 70]
passing_reading_rate = (len(passing_reading) / len(student_data)) * 100

In [4]:
# Create a data frame to show a summary of the district
district_df = pd.DataFrame({
    'Total Schools': [len(school_data)],
    'Total Students': [len(student_data)],
    'Total Budget': [school_data['budget'].sum()],
    'Average Math Score': [student_data['maths_score'].mean()],
    'Average Reading Score': [student_data['reading_score'].mean()],
    '% Passing Math': [passing_math_rate],
    '% Passing Reading' : [passing_reading_rate],
    '% Overall Passing Rate': [(student_data['maths_score'].mean() 
                                + student_data['reading_score'].mean()) / 2]})

# Add some formatting
district_df["Total Students"] = district_df["Total Students"].map("{:,}".format)
district_df["Total Budget"] = district_df["Total Budget"].map("${:,.2f}".format)

# Display the data frame
district_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",70.338192,69.980138,51.460301,50.814399,70.159165


In [5]:
# Determine the School Type
school_types = school_data.set_index(["school_name"])["type"]

# Calculate the total student count
per_school_counts = school_data_complete["school_name"].value_counts()

# Calculate the total school budget and per capita spending
per_school_budget = school_data_complete.groupby(["school_name"]).mean()["budget"]
per_school_capita = per_school_budget / per_school_counts

# Calculate the average test scores
per_school_math = school_data_complete.groupby(["school_name"]).mean()["maths_score"]
per_school_reading = school_data_complete.groupby(["school_name"]).mean()["reading_score"]

# Calculate the passing scores by creating a filtered data frame
school_passing_math = school_data_complete[(school_data_complete["maths_score"] > 70)]
school_passing_reading = school_data_complete[(school_data_complete["reading_score"] > 70)]

per_school_passing_math = school_passing_math.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
per_school_passing_reading = school_passing_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
overall_passing_rate = (per_school_passing_math + per_school_passing_reading) / 2

# Convert to data frame
per_school_summary = pd.DataFrame({"School Type": school_types,
                                   "Total Students": per_school_counts,
                                   "Total School Budget": per_school_budget,
                                   "Per Student Budget": per_school_capita,
                                   "Average Math Score": per_school_math,
                                   "Average Reading Score": per_school_reading,
                                   "% Passing Math": per_school_passing_math,
                                   "% Passing Reading": per_school_passing_reading,
                                   "% Overall Passing Rate": overall_passing_rate})

# Minor data munging
per_school_summary = per_school_summary[["School Type", "Total Students", "Total School Budget", "Per Student Budget",
                                         "Average Math Score", "Average Reading Score", 
                                         "% Passing Math", "% Passing Reading", 
                                         "% Overall Passing Rate"]]
per_school_summary["Total School Budget"] = per_school_summary["Total Students"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the data frame
per_school_summary


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Bailey High School,Government,4976,"$4,976.00",$628.00,72.352894,71.008842,53.496785,50.703376,52.100080
Cabrera High School,Independent,1858,"$1,858.00",$582.00,71.657158,71.359526,51.668461,51.345533,51.506997
Figueroa High School,Government,2949,"$2,949.00",$639.00,68.698542,69.077993,46.354697,47.744998,47.049847
Ford High School,Government,2739,"$2,739.00",$644.00,69.091274,69.572472,47.060971,48.959474,48.010223
Griffin High School,Independent,1468,"$1,468.00",$625.00,71.788147,71.245232,53.133515,52.179837,52.656676
Hernandez High School,Government,4635,"$4,635.00",$652.00,68.874865,69.186408,47.529666,47.939590,47.734628
Holden High School,Independent,427,$427.00,$581.00,72.583138,71.660422,55.035129,53.629977,54.332553
Huang High School,Government,2917,"$2,917.00",$655.00,68.935207,68.910525,47.411724,48.165924,47.788824
Johnson High School,Government,4761,"$4,761.00",$650.00,68.843100,69.039277,47.006931,46.838899,46.922915
Pena High School,Independent,962,$962.00,$609.00,72.088358,71.613306,53.430353,53.326403,53.378378


In [6]:
# Sort and show top five schools
top_schools = per_school_summary.sort_values(["% Overall Passing Rate"], ascending=False)
top_schools.head(5)


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Holden High School,Independent,427,$427.00,$581.00,72.583138,71.660422,55.035129,53.629977,54.332553
Pena High School,Independent,962,$962.00,$609.00,72.088358,71.613306,53.430353,53.326403,53.378378
Griffin High School,Independent,1468,"$1,468.00",$625.00,71.788147,71.245232,53.133515,52.179837,52.656676
Bailey High School,Government,4976,"$4,976.00",$628.00,72.352894,71.008842,53.496785,50.703376,52.100080
Rodriguez High School,Government,3999,"$3,999.00",$637.00,72.047762,70.935984,52.513128,50.687672,51.600400


In [7]:
# Sort and show bottom five schools
bottom_schools = per_school_summary.sort_values(["% Overall Passing Rate"], ascending=True)
bottom_schools.head(5)


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Wilson High School,Independent,2283,"$2,283.00",$578.00,69.170828,68.876916,47.174770,46.430136,46.802453
Johnson High School,Government,4761,"$4,761.00",$650.00,68.843100,69.039277,47.006931,46.838899,46.922915
Figueroa High School,Government,2949,"$2,949.00",$639.00,68.698542,69.077993,46.354697,47.744998,47.049847
Hernandez High School,Government,4635,"$4,635.00",$652.00,68.874865,69.186408,47.529666,47.939590,47.734628
Huang High School,Government,2917,"$2,917.00",$655.00,68.935207,68.910525,47.411724,48.165924,47.788824


In [8]:
# Create data series of scores by year levels using conditionals
year9 = school_data_complete[(school_data_complete["year"] == 9)]
year10 = school_data_complete[(school_data_complete["year"] == 10)]
year11 = school_data_complete[(school_data_complete["year"] == 11)]
year12 = school_data_complete[(school_data_complete["year"] == 12)]

# Group each by school name
year9_scores = year9.groupby(["school_name"]).mean()["maths_score"]
year10_scores = year10.groupby(["school_name"]).mean()["maths_score"]
year11_scores = year11.groupby(["school_name"]).mean()["maths_score"]
year12_scores = year12.groupby(["school_name"]).mean()["maths_score"]

# Combine series into single data frame
scores_by_year = pd.DataFrame({9: year9_scores, 10: year10_scores,
                                11: year11_scores, 12: year12_scores})

# Minor data munging
scores_by_year = scores_by_year[[9, 10, 11, 12]]
scores_by_year.index.name = None

# Display the data frame
scores_by_year

,9,10,11,12
Bailey High School,72.493827,71.897498,72.374900,72.675097
Cabrera High School,72.321970,72.437768,71.008299,70.604712
Figueroa High School,68.477804,68.331586,68.811001,69.325282
Ford High School,69.021609,69.387006,69.248862,68.617811
Griffin High School,72.789731,71.093596,71.692521,71.469178
Hernandez High School,68.586831,68.867156,69.154412,68.985075
Holden High School,70.543307,75.105263,71.640777,73.409639
Huang High School,69.081754,68.533246,69.431345,68.639316
Johnson High School,69.469286,67.990220,68.637730,69.287393
Pena High School,71.996364,72.396000,72.523438,71.187845


In [9]:
# Establish the bins 
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

# Categorize the spending based on the bins
per_school_summary["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=group_names)

spending_math_scores = per_school_summary.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = per_school_summary.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = per_school_summary.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = per_school_summary.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_rate = (spending_math_scores + spending_reading_scores) / 2

# Assemble into data frame
spending_summary = pd.DataFrame({"Average Math Score" : spending_math_scores,
                                 "Average Reading Score": spending_reading_scores,
                                 "% Passing Math": spending_passing_math,
                                 "% Passing Reading": spending_passing_reading,
                                 "% Overall Passing Rate": overall_passing_rate})

# Minor data munging
spending_summary = spending_summary[["Average Math Score", 
                                     "Average Reading Score", 
                                     "% Passing Math", "% Passing Reading",
                                     "% Overall Passing Rate"]]

# Display results
spending_summary


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,71.364587,70.716577,51.566812,50.323634,71.040582
$585-615,72.061215,70.935557,53.149021,50.598466,71.498386
$615-645,70.593378,70.268222,50.074834,49.898085,70.430800
$645-675,68.884391,69.045403,47.316107,47.648138,68.964897


In [10]:
# Establish the bins 
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Categorize the spending based on the bins
per_school_summary["School Size"] = pd.cut(per_school_summary["Total Students"], size_bins, labels=group_names)

# Calculate the scores based on bins
size_math_scores = per_school_summary.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = per_school_summary.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = per_school_summary.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary.groupby(["School Size"]).mean()["% Passing Reading"]
overall_passing_rate = (size_passing_math + size_passing_reading) / 2

# Assemble into data frame
size_summary = pd.DataFrame({"Average Math Score" : size_math_scores,
                             "Average Reading Score": size_reading_scores,
                             "% Passing Math": size_passing_math,
                             "% Passing Reading": size_passing_reading,
                             "% Overall Passing Rate": overall_passing_rate})

# Minor data munging
size_summary = size_summary[["Average Math Score", 
                             "Average Reading Score", 
                             "% Passing Math", "% Passing Reading",
                             "% Overall Passing Rate"]]

# Display results
size_summary


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),72.335748,71.636864,54.232741,53.478190,53.855466
Medium (1000-2000),71.421650,70.720164,51.589692,50.079587,50.834640
Large (2000-5000),69.751809,69.576052,48.568584,48.433759,48.501171


In [11]:
# Type | Average Math Score | Average Reading Score | % Passing Math | % Passing Reading | % Overall Passing Rate

type_math_scores = per_school_summary.groupby(["School Type"]).mean()["Average Math Score"]
type_reading_scores = per_school_summary.groupby(["School Type"]).mean()["Average Reading Score"]
type_passing_math = per_school_summary.groupby(["School Type"]).mean()["% Passing Math"]
type_passing_reading = per_school_summary.groupby(["School Type"]).mean()["% Passing Reading"]
overall_passing_rate = (type_passing_math + type_passing_reading) / 2

# Assemble into data frame
type_summary = pd.DataFrame({"Average Math Score" : type_math_scores,
                             "Average Reading Score": type_reading_scores,
                             "% Passing Math": type_passing_math,
                             "% Passing Reading": type_passing_reading,
                             "% Overall Passing Rate": overall_passing_rate})

# Minor data munging
type_summary = type_summary[["Average Math Score", 
                             "Average Reading Score",
                             "% Passing Math",
                             "% Passing Reading",
                             "% Overall Passing Rate"]]

# Display results
type_summary


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Government,69.834806,69.675929,48.767700,48.719991,48.743845
Independent,71.368822,70.718933,51.698589,50.473056,51.085823


In [ ]:
#1. Based on the math scores, can be concluded that the small and medium size schools have a better performance overall.
#2. In overall, Charter schools have top performance.
#3. School budget is not a determinant factor for better results.